# Processing sentinel-2 data

## DATA MANAGMENT and PREPARATION

In [1]:
import os
from glob import glob
import zipfile
import shutil

# import gui tkinter
from tkinter import Tk
import tkinter.filedialog as tkfd

import logging

In [2]:
logging.basicConfig(format='%(message)s', level='INFO')

### Set input and output directories

**Set input directory**, storing the downloaded Sentinel zipfiles, and the **output directory**, where the processed imagery will be saved.

In [3]:
root = Tk()
input_zip_files = tkfd.askopenfilenames(title = "Select input dataset (zipfiles)", 
                                             initialdir = "C:/", 
                                             filetypes = (("zipfile", "*.zip"),("all files", "*.*")))
output_dir = tkfd.askdirectory(title = "Select output folder", initialdir = "C:/")
bounding_box_flanders = tkfd.askopenfilename(title = "Select input GeoJSON, defining the bounding box of Flanders", 
                                             initialdir = "C:/", 
                                             filetypes = (("GeoJSON", "*.geojson"),("all files", "*.*")))
root.destroy()

In [4]:
input_zip_files

('C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160811T040836_R051_V20160809T105032_20160809T105030.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160812T191647_R094_V20160812T105622_20160812T105622.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160823T181344_R094_V20160822T105652_20160822T110529.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T181148_R008_V20160826T104022_20160826T104023.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160827T195703_R008_V20160826T104022_20160826T104023.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160908T001527_R094_V20160812T105622_20160812T105622.zip')

In [5]:
output_dir

'C:/SS'

In [6]:
bounding_box_flanders

'C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson'

check output_dir for what has been done in the past

In [7]:
validated_zip_files = []
for input_zip_file in input_zip_files:
    date = input_zip_file[-19:-11]
    if date not in os.listdir(output_dir):
        validated_zip_files.append(input_zip_file)
        print(input_zip_file, "will be processed.")
    else:
        print("WARNING: An output folder with date", date, "already exist.", input_zip_file, "is dropped from the selection.")

C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160812T191647_R094_V20160812T105622_20160812T105622.zip will be processed.
C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160908T001527_R094_V20160812T105622_20160812T105622.zip will be processed.


In [8]:
validated_zip_files

['C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160812T191647_R094_V20160812T105622_20160812T105622.zip',
 'C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20160908T001527_R094_V20160812T105622_20160812T105622.zip']

### UNZIP the data

NOTE: Problemen met lengte van de paden in de sentinel datastructuur. Unzippen lukt wel indien niet te ver "verstopt" in de windows tree...

In [9]:
# define and create temporary folder where the files will be unzipped.
extraction_zip_dir = output_dir + "/tempUZ/"
if not os.path.isdir(output_dir + "/tempUZ/"):
    os.mkdir(output_dir + "/tempUZ/")

In [10]:
# unzip files
for validated_zip_file in validated_zip_files:
    z = zipfile.ZipFile(validated_zip_file)
    z.extractall(extraction_zip_dir)

### DELETE useless data

Only granules within or partially within the bounding box of Flanders are kept. All other granules are deleted.

Eight granules overlap with the Flanders bounding box: 
* T31UDT - T31UET - T31UFT - T31UGT
* T31UDS - T31UES - T31UFS - T31UGS

NOTE: Voor de data van juli 2016, +/- 30Gb overbodige data opgekuist op deze manier...

In [11]:
# Eight granules overlap with the Flanders bounding box
selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
# select folders with granules overlapping the bounding box of Flanders...
selected_folders =[]
for folder in glob(os.path.join(extraction_zip_dir, "**", "*/GRANULE/*"), recursive=True):
    for granule in selected_granules:
        if granule in folder:
            selected_folders.append(folder)
# and delete the other granules.
for folder in glob(os.path.join(extraction_zip_dir, "**", "*/GRANULE/*"), recursive=True):
    if folder not in selected_folders:
        shutil.rmtree(folder)

### GROUP by DATE

Rearrange the data and group by date to easily allow mosaicing of imagery.

In [12]:
processing_dir = output_dir + "/" + "P"
if not os.path.isdir(processing_dir):
    os.mkdir(processing_dir)
for filename in os.listdir(extraction_zip_dir):
    date = filename[-20:-12]
    date_folder = processing_dir + "/" + date
    if not os.path.isdir(date_folder):
        os.mkdir(date_folder)
    folder = extraction_zip_dir + "/" + filename
    shutil.move(src = folder, dst = date_folder)

In [13]:
# delete the now empty temporary zip directory
shutil.rmtree(extraction_zip_dir)

## DATA PROCESSING

In [14]:
import rsgislib
from rsgislib import imageutils, RSGISPyUtils
import osgeo
from osgeo import gdal, osr, ogr
import subprocess
import numpy as np
from numpy import *

In [15]:
# Enable GDAL/OGR exceptions and set GDAL data
gdal.UseExceptions()
osgeo.gdal.SetConfigOption("GDAL_DATA", "C:/Users/jeroen.dereu@inbo.be/Software/gdal-2.1.3/data")

In [16]:
#listdir checken
folders_to_be_processed = []
for date_folder in os.listdir(processing_dir): 
    folders_to_be_processed.append(processing_dir + "/" + date_folder + "/")

In [17]:
for date_folder in folders_to_be_processed:
    print(date_folder)

C:/SS/P/20160812/


### Convert JP2 to GeoTiff

Images are converted one by one. Coordinates are transformed from EPSG:32631 to EPSG:4326.

In [18]:
for date_folder in folders_to_be_processed:
    for path in glob(os.path.join(date_folder, "**", "*.jp2"), recursive=True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        if not os.path.isdir(date_folder + "/ind_tifs"):
            os.mkdir(date_folder + "/ind_tifs")
        output_folder = date_folder + "/ind_tifs/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  dstSRS = "EPSG:4326", 
                  srcSRS = "EPSG:32631")

### MOSAIC individual bands

Create a mosaic of the available imagery per day for each of the 13 bands. Also one overview mosaic is produced.

                        Detailed overview of the 13 bands.
|Band name	|Resolution (m)	|Central wavelength (nm)	|Band width (nm)	|Purpose
|:---:|:---:|:---:|:---:|:---:|
|B01	|60	|443	|20	|Aerosol detection|
|B02	|10	|490	|65	|Blue|
|B03	|10	|560	|35	|Green|
|B04	|10	|665	|30	|Red|
|B05	|20	|705	|15	|Vegetation classification|
|B06	|20	|740	|15	|Vegetation classification|
|B07	|20	|783	|20	|Vegetation classification|
|B08	|10	|842	|115	|Near infrared|
|B08A	|20	|865	|20	|Vegetation classification|
|B09	|60	|945	|20	|Water vapour|
|B10	|60	|1375	|30	|Cirrus|
|B11	|20	|1610	|90	|Snow / ice / cloud discrimination|
|B12	|20	|2190	|180	|Snow / ice / cloud discrimination|


In [19]:
for date_folder in folders_to_be_processed:
    date = date_folder[-9:-1]
    selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
    selected_images = []
    B01 = []
    B02 = []
    B03 = []
    B04 = []
    B05 = []
    B06 = []
    B07 = []
    B08 = []
    B8A = []
    B09 = []
    B10 = []
    B11 = []
    B12 = []
    overview = []
    for path in glob(os.path.join(date_folder, "**", "*.jp2"), recursive=True):
        for granule in selected_granules:
            if granule in path:
                selected_images.append(path)
    for image in selected_images:
        if image.endswith("B01.jp2"):
            B01.append(image)
        elif image.endswith("B02.jp2"):
            B02.append(image)
        elif image.endswith("B03.jp2"):
            B03.append(image)
        elif image.endswith("B04.jp2"):
            B04.append(image)
        elif image.endswith("B05.jp2"):
            B05.append(image)
        elif image.endswith("B06.jp2"):
            B06.append(image)
        elif image.endswith("B07.jp2"):
            B07.append(image)
        elif image.endswith("B08.jp2"):
            B08.append(image)        
        elif image.endswith("B8A.jp2"):
            B8A.append(image)
        elif image.endswith("B09.jp2"):
            B09.append(image)
        elif image.endswith("B10.jp2"):
            B10.append(image)        
        elif image.endswith("B11.jp2"):
            B11.append(image)      
        elif image.endswith("B12.jp2"):
            B12.append(image)         
        else:
            overview.append(image)
    if not os.path.isdir(date_folder + "/merged_bands"):
        os.mkdir(date_folder + "/merged_bands")
    output_folder = date_folder + "/merged_bands/"
    output_B01 = output_folder + date + "_B01" + ".tif"
    output_B02 = output_folder + date + "_B02" + ".tif"
    output_B03 = output_folder + date + "_B03" + ".tif"
    output_B04 = output_folder + date + "_B04" + ".tif"
    output_B05 = output_folder + date + "_B05" + ".tif"
    output_B06 = output_folder + date + "_B06" + ".tif"
    output_B07 = output_folder + date + "_B07" + ".tif"
    output_B08 = output_folder + date + "_B08" + ".tif"
    output_B8A = output_folder + date + "_B8A" + ".tif"
    output_B09 = output_folder + date + "_B09" + ".tif"   
    output_B10 = output_folder + date + "_B10" + ".tif"
    output_B11 = output_folder + date + "_B11" + ".tif"
    output_B12 = output_folder + date + "_B12" + ".tif"
    output_overview = output_folder + date + "_overview" + ".tif"
    gdal.Warp(destNameOrDestDS = output_B01, 
              srcDSOrSrcDSTab = B01, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B02, 
              srcDSOrSrcDSTab = B02, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B03, 
              srcDSOrSrcDSTab = B03, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)        
    gdal.Warp(destNameOrDestDS = output_B04, 
              srcDSOrSrcDSTab = B04, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)           
    gdal.Warp(destNameOrDestDS = output_B05, 
              srcDSOrSrcDSTab = B05, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)             
    gdal.Warp(destNameOrDestDS = output_B06, 
              srcDSOrSrcDSTab = B06, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B07, 
              srcDSOrSrcDSTab = B07, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B08, 
              srcDSOrSrcDSTab = B08, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B8A, 
              srcDSOrSrcDSTab = B8A, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B09, 
              srcDSOrSrcDSTab = B09, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B10, 
              srcDSOrSrcDSTab = B10, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B11, 
              srcDSOrSrcDSTab = B11, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_B12, 
              srcDSOrSrcDSTab = B12, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    gdal.Warp(destNameOrDestDS = output_overview, 
              srcDSOrSrcDSTab = overview, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = bounding_box_flanders)
    
# NOTE: bounding_box_flanders ergens via relatief pad opslaan, of een soort temp json aanmaken.... even nakijken en betere oplossing zoeken.

### STACK bands

Stack a series of bands in one GeoTiff, e.g. RGB bands...

In [20]:
for date_folder in folders_to_be_processed:
    RGB = []
    for path in glob(os.path.join(date_folder, "**", "merged_bands", "*tif"), recursive=True):
        if path.endswith("B04.tif"):
            RGB.append(path)
        if path.endswith("B03.tif"):
            RGB.append(path) 
        if path.endswith("B02.tif"):
            RGB.append(path)
    bandNamesList = ["Red", "Green", "Blue"]
    if not os.path.isdir(date_folder + "/stacked/"):
        os.mkdir(date_folder + "/stacked/")
    date = date_folder[-9:-1]
    outputImage = date_folder + "/stacked/" + date + "_RGB.tif"
    gdalFormat = "GTiff"
    dataType = rsgislib.TYPE_16UINT
    imageutils.stackImageBands(RGB, bandNamesList, outputImage, None, 0, gdalFormat, dataType)

### NDVI - Normalised Difference Vegetation Index

**NDVI = (NIR - R) / (NIR) + R**

    NIR: Near Infra Red - Sentinelsat band 8
    R: Visible Red - Sentinelsat band 4

In [21]:
for date_folder in folders_to_be_processed:
    for path in glob(os.path.join(date_folder, "**", "merged_bands", "*tif"), recursive=True):
        if path.endswith("B04.tif"):
            B04 = gdal.Open(path)
        if path.endswith("B08.tif"):
            B08 = gdal.Open(path)
    red = B04.ReadAsArray()
    nir = B08.ReadAsArray()
    check = np.logical_and(red > 0, nir > 0)
    ndvi = np.where(check, (nir - red) / (nir + red), -999)
    geo = B08.GetGeoTransform()
    proj = B08.GetProjection()
    shape = red.shape
    driver = gdal.GetDriverByName("GTiff")
    # define output filename and location
    if not os.path.isdir(date_folder + "/ndvi/"):
        os.mkdir(date_folder + "/ndvi/")
    date = date_folder[-9:-1]
    ndvi_file = date_folder + "/ndvi/" + date + "_ndvi.tif"
    # create and write the nvdi-file
    dst_ds = driver.Create(ndvi_file, shape[1], shape[0], 1, gdal.GDT_Float32)
    dst_ds.SetGeoTransform(geo)
    dst_ds.SetProjection(proj)
    dst_ds.GetRasterBand(1).WriteArray(ndvi)
    dst_ds = None
    B04 = None
    B08 = None

C:\Users\jeroen.dereu@inbo.be\AppData\Local\Continuum\Anaconda3\envs\osgeoenv\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


### DATA MANAGEMENT

Delete raw data

In [22]:
for path in glob(os.path.join(processing_dir, "**", "S2A*.SAFE"), recursive=True):
    shutil.rmtree(path)

Move the processed data to the output directory

In [23]:
for processed_dataset in os.listdir(processing_dir):
    processed_dataset_folder = processing_dir + "/" + processed_dataset
    shutil.move(src = processed_dataset_folder, dst = output_dir)

Delete the now empty temporary processing directory

In [24]:
shutil.rmtree(processing_dir)